## Data Cleaning and Preprocessing for creating turnout model for Alamance County, NC

In [13]:
# written by @caramnix 12.27.24 
# goal: Data cleaning/preprocessing

In [14]:
# imports
import pandas as pd
import numpy as np 
import os

In [31]:
os.chdir("/Users/caranix/Documents/GitHub/AlamanceTurnout/")

## Cut voter files to Alamance County

In [ ]:
# read in voter files (~12GB) and cut to only voters in Alamance county 
data2016 = pd.read_csv('/Users/caranix/Documents/GitHub/AlamanceTurnout/VR_Snapshot_20161108.txt', sep='\t', encoding = "utf-16", on_bad_lines='warn')
data2020 = pd.read_csv('/Users/caranix/Documents/GitHub/AlamanceTurnout/VR_Snapshot_20201103.txt', sep='\t', encoding = "utf-16", on_bad_lines='warn')


In [18]:
alamance2016 = data2016[data2016["county_id"] ==1]
alamance2020 = data2020[data2020["county_id"] ==1]

In [19]:
alamance2016.to_csv('NC_Voterfiles_Alamance/VR_Snapshot_20161108.csv', index=False)
alamance2020.to_csv('NC_Voterfiles_Alamance/VR_Snapshot_20201103.csv', index=False)

In [26]:
# read in Alamance data from 2016 -- voter file 

vf_2016= pd.read_csv("VR_Snapshot_20161108.csv")

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_3390/2882072387.py:3: DtypeWarning: Columns (25,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  vf_2016= pd.read_csv("VR_Snapshot_20161108.csv")


In [28]:
vf_2016.shape 
# 213,664 people on file 

(213664, 90)

In [27]:
# Appears as if inactive/removed folks are on voter file, will remove them for modeling since they will not 
# turnout if voter reg status is not active 
np.unique(vf_2016['voter_status_desc'])

array(['ACTIVE', 'DENIED', 'INACTIVE', 'REMOVED', 'TEMPORARY'],
      dtype=object)

In [ ]:
vf_2016_a = vf_2016[vf_2016["voter_status_desc"] =="ACTIVE"]

In [30]:
# list all columns availible
vf_2016_a.columns

Index(['snapshot_dt', 'county_id', 'county_desc', 'voter_reg_num', 'ncid',
       'status_cd', 'voter_status_desc', 'reason_cd',
       'voter_status_reason_desc', 'absent_ind', 'name_prefx_cd', 'last_name',
       'first_name', 'midl_name', 'name_sufx_cd', 'house_num', 'half_code',
       'street_dir', 'street_name', 'street_type_cd', 'street_sufx_cd',
       'unit_designator', 'unit_num', 'res_city_desc', 'state_cd', 'zip_code',
       'mail_addr1', 'mail_addr2', 'mail_addr3', 'mail_addr4', 'mail_city',
       'mail_state', 'mail_zipcode', 'area_cd', 'phone_num', 'race_code',
       'race_desc', 'ethnic_code', 'ethnic_desc', 'party_cd', 'party_desc',
       'sex_code', 'sex', 'age', 'birth_place', 'registr_dt', 'precinct_abbrv',
       'precinct_desc', 'municipality_abbrv', 'municipality_desc',
       'ward_abbrv', 'ward_desc', 'cong_dist_abbrv', 'cong_dist_desc',
       'super_court_abbrv', 'super_court_desc', 'judic_dist_abbrv',
       'judic_dist_desc', 'NC_senate_abbrv', 'NC_sena

In [122]:
# only keep some of the columsn avaible which may be useful for modeling -- note this selection is not extensive
cols_to_keep= ['snapshot_dt', 'county_id', 'voter_reg_num', 'ncid', 'race_code','ethnic_code', 'area_cd', 'party_cd', 'sex_code', 'age', 'birth_place', 'registr_dt','precinct_abbrv','precinct_desc', 'age_group', 'mail_zipcode']


In [123]:
vf_2016_a_cut = vf_2016_a[cols_to_keep]

## Read in Vote History Data 

In [271]:
# read in vote history data
# get rid of data > 2020, as we are trying to predict turnout in 2020

vote_history_alamance= pd.read_csv("Alamance Vote History/ncvhis1.txt",  sep='\t', on_bad_lines='warn')

elections_to_keep= ['11/03/2020', '06/23/2020', '03/03/2020', 
                    '11/05/2019', '10/08/2019', '09/10/2019', '07/09/2019', '05/14/2019', '04/30/2019',
                    '11/06/2018', '06/26/2018', '05/08/2018', 
                    '11/07/2017', '10/10/2017', '09/12/2017', 
                    '11/08/2016', '06/07/2016', '03/15/2016',
                    '11/03/2015', '10/06/2015', '09/15/2015'] 

vote_history_alamance = vote_history_alamance[vote_history_alamance['election_lbl'].isin(elections_to_keep)]


In [272]:
# going to want column for each of these elections, i.e. 1 if voted, 0 if not...2020_g, 2020_p 
vote_history_alamance['election_lbl'].unique() 

array(['11/03/2020', '03/03/2020', '11/08/2016', '03/15/2016',
       '11/06/2018', '11/07/2017', '05/08/2018', '06/07/2016',
       '11/05/2019', '11/03/2015', '10/08/2019', '10/10/2017',
       '10/06/2015', '09/10/2019', '09/12/2017', '05/14/2019',
       '04/30/2019', '07/09/2019', '06/23/2020', '06/26/2018',
       '09/15/2015'], dtype=object)

In [273]:
# g= general 
# p = primary 
# pr = primary runoff  (?: don't want people w/o runoff to be penalized)
# o = other form of election 
d = {
    '11/03/2020': '2020_g',
    '06/23/2020': '2020_pr',
    '03/03/2020': '2020_p',
    
    '11/05/2019': '2019_g',
    '10/08/2019': '2019_o',
    '04/30/2019': '2019_o',
    '07/09/2019': '2019_o',
    
    '11/06/2018': '2018_g', 
    '06/26/2018': '2018_pr',
    '05/08/2018': '2018_p',
    
    '11/07/2017': '2017_g',
    '10/10/2017': '2017_o',
    '09/12/2017': '2017_o',
    
    '11/08/2016': '2016_g',
    '06/07/2016': '2016_pr',
    '03/15/2016': '2016_p',
    
    '11/03/2015': '2015_g',
    '10/06/2015': '2015_o',
    '09/15/2015': '2015_o'
}
vote_history_alamance['election_year_type'] = vote_history_alamance['election_lbl'].map(d)

In [274]:
vote_history_alamance['voting_method'].unique() # --> Election Day, Absentee, Early Voting, Provisional/Transfer

array(['ABSENTEE ONESTOP', 'ELECTION DAY IN-PERSON', 'ABSENTEE BY MAIL',
       'ABSENTEE CURBSIDE', 'PROVISIONAL', 'ELECTION DAY CURBSIDE',
       'TRANSFER'], dtype=object)

In [275]:
# 1 - early
# 0 - in person/ provisional

d2= {'ABSENTEE ONESTOP': 1, 
     'ELECTION DAY IN-PERSON': 0,
     'EARLY VOTING IN-PERSON': 1,
     'ABSENTEE BY MAIL': 1,
     'PROVISIONAL' : 0, 
     'ABSENTEE CURBSIDE': 1, 
     'EARLY VOTING CURBSIDE' : 1,
     'ELECTION DAY CURBSIDE': 0,
     'TRANSFER': 0}
vote_history_alamance['vote_method'] = vote_history_alamance['voting_method'].map(d2)

In [204]:
# Filter out rows with election dates in 2020 -- note: this needs to be done on the 2020 voter file we are predicting
# This is because we want to use it as a predictor, i.e. have they ever voted early in the past? 

filtered_df = vote_history_alamance[~vote_history_alamance['election_year_type'].isin(['2020_g', '2020_pr'])]

# aggregate
result = filtered_df[['ncid', 'vote_method']].groupby('ncid', as_index=False).max()

voted_early_df_before_2020= result.rename({'vote_method': 'voted_early_prior_to_2020_g'}, axis=1)

In [277]:
# Filter out rows with election dates >= 2016 general
# This is because we want to use it as a predictor, i.e. have they ever voted early in the past? 

filtered_df2 = vote_history_alamance[vote_history_alamance['election_year_type'].isin(['2015_g', '2015_o', '2016_p', '2016_pr'])]

# aggregate
result2 = filtered_df2[['ncid', 'vote_method']].groupby('ncid', as_index=False).max()

voted_early_df_before_2016= result2.rename({'vote_method': 'voted_early_prior_to_2016_g'}, axis=1)


In [278]:
# need to transform data from long to wide -- 
# i.e. one row for each voter reg #
# county_id, county_desc, voter_reg_num, if 11/03/2020 General exists 

vote_history_alamance['participated'] = 1

# Pivot wider
pivot_df = vote_history_alamance.pivot_table(
    index=['ncid', 'voter_reg_num', 'county_id', 'county_desc'], #, 'pct_description', 'pct_label'],
    columns='election_year_type',
    values='participated', 
    aggfunc='max',
    fill_value=0
)

# note: wouldn't include movers's history then if they moved counties....room for improvement later 

In [279]:
vote_history_alamance_wider= pivot_df.reset_index()

In [280]:
vote_history_alamance_wider

election_year_type,ncid,voter_reg_num,county_id,county_desc,2015_g,2015_o,2016_g,2016_p,2016_pr,2017_g,2017_o,2018_g,2018_p,2018_pr,2019_g,2019_o,2020_g,2020_p,2020_pr
0,AA100000,9050398,1,ALAMANCE,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
1,AA100006,9050405,1,ALAMANCE,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0
2,AA100007,9050406,1,ALAMANCE,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0
3,AA100009,9050408,1,ALAMANCE,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0
4,AA10001,1242500,1,ALAMANCE,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102130,ES22005,9197961,1,ALAMANCE,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
102131,ES23246,9178489,1,ALAMANCE,0,0,1,1,0,0,0,1,0,0,0,0,1,1,0
102132,ES25042,9241785,1,ALAMANCE,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
102133,ES25192,9198550,1,ALAMANCE,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [281]:
alamace_county_merged= pd.merge(vf_2016_a_cut, vote_history_alamance_wider, on='voter_reg_num', how='left')


In [120]:
# note: since vote history only goes through ~2015, did not include voter files from previous elections, since would not have any validation column/data! 

In [284]:
alamace_county_merged= alamace_county_merged.rename({'ncid_x': 'ncid'}, axis=1)
alamace_county_merged= pd.merge(alamace_county_merged, voted_early_df_before_2016, left_on='ncid', right_on='ncid', how='left')

In [285]:
#10k voted early prior to 2016 general -- might not be enough data here to use, might over predict voting based on this! 
np.sum(alamace_county_merged['voted_early_prior_to_2016_g'])

9008.0

In [ ]:
# next steps: recoding variables

# why ever absentee-- if voted absentee/early, seems likley they'd vote again more than people who voted on eday 

In [286]:
alamace_county_merged['registr_dt'] = pd.to_datetime(alamace_county_merged['registr_dt'])

# Extract the year from the 'election_date' column
alamace_county_merged['registration_year'] = alamace_county_merged['registr_dt'].dt.year

In [287]:
alamace_county_merged['in_cycle_reg'] = alamace_county_merged['registration_year'] == 2016

In [288]:
np.sum(alamace_county_merged['in_cycle_reg'])

12022

In [291]:
alamace_county_merged.to_csv("alamance_2016_vf_data.csv", index=False)


In [290]:
# okay, so what do NA's mean here -- well 
# to get 2016_g we used vote history data set, i.e. for each ncid we have record of if they voted in the general 
# we joined this to the 2016 voter file (left), so NA's mean they were on the voter file but DID NOT match to the 2016 general election (or any election) 
# so NA's indicate people on the voter file who DID NOT VOTE, since 2016_g eas created from vote history data set. 

alamace_county_merged['2016_g'].unique()

array([ 1., nan,  0.])

 ## Now repeat data cleaning/ preprocessing for 2020 voter file 
 

In [ ]:
# heads up, not commenting this as fully because it is the same process as what I did above! 

In [292]:
vf_2020= pd.read_csv("/Users/caranix/Documents/murmuration/NC_Voterfiles_Alamance/VR_Snapshot_20201103.csv")

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_47245/3868972654.py:1: DtypeWarning: Columns (25,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  vf_2020= pd.read_csv("/Users/caranix/Documents/murmuration/NC_Voterfiles_Alamance/VR_Snapshot_20201103.csv")


In [296]:
vf_2020_a = vf_2020[vf_2020["voter_status_desc"] =="ACTIVE"]

vf_2020_a_cut = vf_2020_a[cols_to_keep]

alamace_county_merged_2020= pd.merge(vf_2020_a_cut, vote_history_alamance_wider, on='voter_reg_num', how='left')

alamace_county_merged_2020= alamace_county_merged_2020.rename({'ncid_x': 'ncid'}, axis=1)

alamace_county_merged_2020= pd.merge(alamace_county_merged_2020, voted_early_df_before_2020, left_on='ncid', right_on='ncid', how='left')

np.sum(alamace_county_merged_2020['voted_early_prior_to_2020_g'])

In [ ]:
# nice a lot of people voted early in past 

In [300]:
# feature: registration year in-cycle
alamace_county_merged_2020['registr_dt'] = pd.to_datetime(alamace_county_merged_2020['registr_dt'])

# Extract the year from the 'election_date' column
alamace_county_merged_2020['registration_year'] = alamace_county_merged_2020['registr_dt'].dt.year

In [301]:
alamace_county_merged_2020['in_cycle_reg'] = alamace_county_merged_2020['registration_year'] == 2020

In [302]:
np.sum(alamace_county_merged_2020['in_cycle_reg'])

14891

In [303]:
alamace_county_merged_2020.to_csv("alamance_2020_vf_data.csv", index=False)


In [494]:
# ~20k leaving file between 2016 and 2020 
len(list(set(alamace_county_merged['ncid']) - set(alamace_county_merged_2020['ncid'])))

20231

In [493]:
# ~35k people joining file between 2016 and 2020 
len(list(set(alamace_county_merged_2020['ncid']) - set(alamace_county_merged['ncid'])))

35783

In [505]:
group_counts = alamace_county_merged['race_code'].value_counts()
percentages = group_counts / group_counts.sum() * 100
percentages

race_code
W    72.307604
B    20.223838
U     3.995616
O     1.874928
M     0.698050
A     0.629976
I     0.269990
Name: count, dtype: float64

In [506]:
group_counts = alamace_county_merged['age_group'].value_counts()
percentages = group_counts / group_counts.sum() * 100
percentages

age_group
Age 41 - 65    44.622130
Age Over 66    23.140648
Age 26 - 40    20.291912
Age 18 - 25    11.945310
Name: count, dtype: float64

In [507]:
group_counts = alamace_county_merged['party_cd'].value_counts()
percentages = group_counts / group_counts.sum() * 100
percentages

party_cd
DEM    38.661590
REP    34.098304
UNA    26.805123
LIB     0.434983
Name: count, dtype: float64

In [508]:
group_counts = alamace_county_merged['sex_code'].value_counts()
percentages = group_counts / group_counts.sum() * 100
percentages

sex_code
F    54.202146
M    43.434868
U     2.362986
Name: count, dtype: float64

In [500]:
## now for 2020 
group_counts = alamace_county_merged_2020['race_code'].value_counts()
percentages = group_counts / group_counts.sum() * 100
percentages

race_code
W    65.106337
B    19.251238
U    11.485786
O     2.420223
A     0.820763
M     0.672067
I     0.236740
P     0.006848
Name: count, dtype: float64

In [502]:
group_counts = alamace_county_merged_2020['age_group'].value_counts()
percentages = group_counts / group_counts.sum() * 100
percentages

age_group
Age 41 - 65    41.240633
Age Over 66    23.010702
Age 26 - 40    21.738960
Age 18 - 25    14.009704
Name: count, dtype: float64

In [503]:
group_counts = alamace_county_merged_2020['party_cd'].value_counts()
percentages = group_counts / group_counts.sum() * 100
percentages

party_cd
DEM    35.171490
REP    33.554421
UNA    30.624523
LIB     0.555653
CST     0.063587
GRE     0.030326
Name: count, dtype: float64

In [504]:
group_counts = alamace_county_merged_2020['sex_code'].value_counts()
percentages = group_counts / group_counts.sum() * 100
percentages

sex_code
F    50.119348
M    40.727045
U     9.153607
Name: count, dtype: float64

## Feature: get precinct-level turnout from last general election 

In [35]:
# read in total votes by precinct from 2012 
precinct_turnout_2012= pd.read_csv("/Users/caranix/Documents/GitHub/AlamanceTurnout/results_pct_20121106.txt")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/caranix/Documents/GitHub/AlamanceTurnout/results_pct_20121106.txt'

In [311]:
pto_2012= precinct_turnout_2012[precinct_turnout_2012["county"] == "ALAMANCE"]
pto_2012_alamance = pto_2012[pto_2012["contest"] == "PRESIDENT AND VICE PRESIDENT OF THE UNITED STATES"]

In [314]:
np.sum(pto_2012_alamance['total votes']) # 67k votes in county 

67776

In [318]:
#len(pto_2012_alamance['precinct'].unique())

40

In [443]:
#pto_2012_alamance['precinct'].unique()

In [328]:
# extract precinct name

pto_2012_alamance['precinct_cleaned']= pto_2012_alamance['precinct'].str.split("_").str[1]

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_47245/3202452234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pto_2012_alamance['precinct_cleaned']= pto_2012_alamance['precinct'].str.split("_").str[1]


In [339]:
# get count of voters within each precinct 
voter_count_2012= pto_2012_alamance[['precinct_cleaned','total votes']].groupby('precinct_cleaned').sum('total_votes')

In [444]:
#pto_2012_alamance['precinct_cleaned'].unique()

In [424]:
# read in voter file from 2012, i.e. how many eligible voters there were 
vf_2012= pd.read_csv("/Users/caranix/Documents/murmuration/NC_Voterfiles_Alamance/VR_Snapshot_20121106.csv")

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_47245/1366575137.py:1: DtypeWarning: Columns (25,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  vf_2012= pd.read_csv("/Users/caranix/Documents/murmuration/NC_Voterfiles_Alamance/VR_Snapshot_20121106.csv")


In [425]:
vf_2012_a = vf_2012[vf_2012["voter_status_desc"] =="ACTIVE"]

In [431]:
vf_2012_a_cut= vf_2012_a[['precinct_abbrv', 'precinct_desc', 'voter_reg_num']]

In [445]:
#vf_2012_a_cut['precinct_desc'].unique()

In [434]:
# what is missing? Mellvile 4 - will assign mean turnout instead! 
list(set(vf_2012_a_cut['precinct_desc']) - set(pto_2012_alamance['precinct_cleaned'])) 

['MELVILLE 4']

In [435]:
# get # of registered voters 
registered_voter_count= vf_2012_a_cut[['voter_reg_num','precinct_desc']].groupby('precinct_desc').count()

In [436]:
registered_voter_count= registered_voter_count.reset_index()

In [437]:
# combine two data sets, to have number of voters v number of registered voters 
precinct_turnout_2012= pd.merge(registered_voter_count, voter_count_2012, right_on = "precinct_cleaned", left_on= 'precinct_desc', how = 'left')

In [472]:
precinct_turnout_2012["turnout_pct"]= precinct_turnout_2012['total votes']/ precinct_turnout_2012['voter_reg_num']

# replace NA with mean turnout note: this only impacts 5 people 
precinct_turnout_2012.fillna(precinct_turnout_2012['turnout_pct'].mean(), inplace=True)

precinct_turnout_2012['year'] = 2012 

precinct_turnout_2012.head()

,precinct_desc,voter_reg_num,precinct_cleaned,total votes,turnout_pct,year
0,ALBRIGHT,2681,ALBRIGHT,2043.0,0.762029,2012
1,BOONE 5,3524,BOONE 5,2291.0,0.650114,2012
2,BURLINGTON 10,1463,BURLINGTON 10,1108.0,0.757348,2012
3,BURLINGTON 4,1678,BURLINGTON 4,1283.0,0.764601,2012
4,BURLINGTON 5,1849,BURLINGTON 5,1396.0,0.755003,2012


In [484]:
alamace_county_merged= pd.merge(alamace_county_merged, precinct_turnout_2012, left_on='precinct_desc' , right_on= 'precinct_cleaned', how='left')


In [485]:
# now add to alamance_2016_vf_data

alamace_county_merged.to_csv("alamance_2016_vf_data.csv", index=False)


### now do this all again for 2016! 


In [369]:
precinct_turnout_2016= pd.read_csv("/Users/caranix/Documents/murmuration/NC precinct-level results/results_pct_20161108.txt"
, sep='\t')

In [377]:
precinct_turnout_2016= precinct_turnout_2016[precinct_turnout_2016['Contest Name']=='US PRESIDENT']
pto_2016_alamance = precinct_turnout_2016[precinct_turnout_2016["County"] == "ALAMANCE"]

In [447]:
# pto_2016_alamance

In [379]:
np.sum(pto_2016_alamance['Total Votes'])

71157

In [383]:
pto_2016_alamance['Precinct'].unique()

array(['09S', '06E', '13', '12W', '05', '02', '03W', '03N2', '11', '12E',
       '06N', '06S', '063', '10S', '127', '125', '103', '1210', '064',
       '03C', '09N', '129', '126', '07', '04', '12N', '12S', '124', '01',
       '128', '10N', '03S', '03N', '035', '06W', '08N', '08S'],
      dtype=object)

In [391]:
pto_2016_alamance['Precinct']= pto_2016_alamance['Precinct'].astype(str)

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_47245/2013523679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pto_2016_alamance['Precinct']= pto_2016_alamance['Precinct'].astype(str)


In [449]:
registered_voter_count_2016= vf_2016_a_cut[['voter_reg_num','precinct_desc']].groupby('precinct_desc').count()

In [384]:
# read in mapping code --> name 

id_name= pd.read_csv("precinct_id_to_name_2020.csv")

In [393]:
id_name= id_name[['prec_id', "enr_desc"]]
id_name['enr_desc']= id_name['enr_desc'].astype(str)

In [418]:
# changing 10S1 to 10S to match desitnation 
id_name.loc[id_name['prec_id']=='10S1','prec_id'] = '10S'

In [420]:
pto_2016_alamance_v2= pd.merge(pto_2016_alamance, id_name, left_on='Precinct', right_on='prec_id', how='left' )

In [451]:
# pto_2016_alamance_v2[pto_2016_alamance_v2['enr_desc'].isna()]

In [452]:
# closest match is 10S1= "SOUTH MELVILLE"

In [468]:
pto_2016_alamance_v3= pto_2016_alamance_v2.groupby("enr_desc").sum('Total Votes').reset_index()

In [470]:
# combine datasets! 
pto_2016_alamance_v4= pd.merge(pto_2016_alamance_v3, registered_voter_count_2016, left_on='enr_desc', right_on='precinct_desc', how='left' )



In [474]:
pto_2016_alamance_v4["turnout_pct"]= pto_2016_alamance_v4['Total Votes']/ pto_2016_alamance_v4['voter_reg_num']

# replace NA with mean turnout note: this only impacts 5 people 
pto_2016_alamance_v4.fillna(pto_2016_alamance_v4['turnout_pct'].mean(), inplace=True)

pto_2016_alamance_v4['year'] = 2016

pto_2016_alamance_v4.head()

,enr_desc,Contest Group ID,Vote For,Election Day,One Stop,Absentee by Mail,Provisional,Total Votes,Unnamed: 14,voter_reg_num,turnout_pct,year
0,ALBRIGHT,5005,5,913,1179,88,5,2185,0.0,2522,0.866376,2016
1,BOONE 5,5005,5,1074,1388,111,11,2584,0.0,3311,0.780429,2016
2,BURLINGTON 10,5005,5,502,672,45,11,1230,0.0,1558,0.789474,2016
3,BURLINGTON 4,5005,5,491,711,57,4,1263,0.0,1524,0.828740,2016
4,BURLINGTON 5,5005,5,519,859,67,7,1452,0.0,1724,0.842227,2016


In [480]:
#alamace_county_merged_2020['precinct_desc']

In [487]:
alamace_county_merged_2020= pd.merge(alamace_county_merged_2020, pto_2016_alamance_v4, left_on='precinct_desc' , right_on= 'enr_desc', how='left')


In [488]:
alamace_county_merged_2020.to_csv("alamance_2020_vf_data.csv", index=False)
